## CNN para previsão de fraturas

**Acerca do Dataset**
- Este conjunto de dados é composto por imagens de raios X fraturadas e não fraturadas de várias articulações. A tarefa é construir um classificador de imagens para detectar fraturas em determinada imagem de raio-X. Este conjunto de dados é composto por diferentes articulações nas extremidades superiores. O isolamento de articulações individuais é recomendado para melhorar o desempenho dos classificadores.
- 8863 instâncias para treino
- 600 instâncias para teste

### Imports

In [170]:
import torch
import numpy as np

from torch.utils.data import DataLoader

from torchvision.datasets import ImageFolder

from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from torchvision.transforms import Normalize
from torchvision.transforms import Resize

### 0. Preparar o Setup

In [171]:
PATH_TRAIN = './Dataset/train'
PATH_TEST = './Dataset/val'

BATCH_SIZE = 32

In [172]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cpu


### 1. Preparar os dados

In [173]:
transform = Compose(
        [ToTensor(),
         Resize((224, 224)),
         Normalize(mean=(0.1307,), std=(0.3081,)) # TODO: variar este valor para perceber a diferença 
        ])

In [174]:
# Definir a seed

seed = 2024

torch.manual_seed(seed)
np.random.seed(seed)

# Importar dataset

train_dataset = ImageFolder(PATH_TRAIN, transform=transform)
val_dataset = ImageFolder(PATH_TEST, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

### 1.1 Visualizar os dados

In [175]:
# TODO: fazer uam mistura entre o que está nos slides e no notebook do kaggle
